In [74]:
import os
import numpy as np
import pandas as pd
import music21 as m21
from typing import List
from itertools import product
from collections import defaultdict
from scipy.sparse import dok_matrix
from easy_ml.tools.util import download_from_gcs

In [2]:
mkdir /home/jovyan/persistent_data/data/mozart

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


mkdir: cannot create directory ‘/home/jovyan/persistent_data/data/mozart’: File exists


In [3]:
DATA_DIR = '/home/jovyan/persistent_data/data/mozart/'

In [4]:
download_from_gcs(bucket_name='midi-files',
                  prefix='collections/k_collection/mozart',
                  local_fs_loc=DATA_DIR)

In [4]:
midi_files = os.listdir(DATA_DIR)

## Note Encoding Schemes
* m-Bar Encodings: Similar to a term-document frequency matrix. Encode a multinomial distribution of notes over a sequence of m measures
    * Parts together in their
    * Pitches encoded as their numerical value, rests encoded as 0
    * Pitches encoded as their numerical value, no rests
* m-bar time series: Each part is an m-bar time series over 127 notes
    * Pitches encoded as their numerical value, rests encoded as 0 (only sensible way in this scenario)
* n-Note Encodings: Similar to n-grams; sequences of m measures will be encoded into $127^n$ features
    * Pitches encoded as their numerical value, rests encoded as 0
    * Pitches encoded as their numerical value, no rests
* n-Note Pitch-Difference Encodings: Similar to above, but instead of mapping each feature distinct pitch as a feature, the difference between subsequent pitches will be analyzed to try to capture melodic patterns
    * Pitches encoded as their numerical value, rests encoded as 0? (probably not sensible in this scenario)
    * Pitches encoded as their numerical value, no rests
    * Do we add durations (like in A Comparison of Statistical Approaches to Symbolic Genre Recognition)

**NOTE** Generate the full feature set by iterating through the entire scale of notes

### n-Note Encodings

In [8]:
s = m21.converter.parse(DATA_DIR + midi_files[12])

In [9]:
def window_gen(sequence, n):
    windowed_iterator = []
    low = 0
    high = n
    for note in sequence:
        window = sequence[low:high]
        if len(window) < n:
            break
        low += 1
        high += 1
        yield tuple(window)

In [10]:
def get_n_note_sequence(part: m21.stream.Part,
                        window_size: int = 2):
    just_notes = part.recurse().notes \
        .getElementsNotOfClass(m21.chord.Chord)
    n_note_sequences = []
    for note_seq in window_gen(just_notes, window_size):
        n_note_sequences.append("|".join([note.nameWithOctave for note in note_seq]))
    return n_note_sequences

In [11]:
def get_sequences_from_stream(stream: m21.stream,
                              window_size: int = 2):
    parts = s.getElementsByClass(m21.stream.Part)
    sequences_list = []
    for part in parts:
        sequences = get_n_note_sequence(part, window_size)
        for seq in sequences:
            sequences_list.append(seq)
    return sequences_list

In [ ]:
def sequence_encoder(seq: List[str]):
    d = defaultdict(float)
    for entry in seq:
        d[entry] += 1.
    return d

In [37]:
mozart_grams = []
for i, midi in enumerate(midi_files[:50]):
    if i % 10 == 0:
        print(i)
    s = m21.converter.parse(DATA_DIR + midi)
    mozart_grams.append(get_sequences_from_stream(s))

In [20]:
class MidiFeatureVector(object):
    def __init__(self, note_sequence):
        self.note_sequence = note_sequence

In [178]:
class MidiFeatureCorpus(object):
    def __init__(self, path: str):
        self.path = path + "/"
        self.files = os.listdir(self.path)
        self.corpus_ = []
        self.note_sequence_set = self._initialize_note_sequence_set()
        self.sparse_matrix = dok_matrix((len(self.files),
                                         len(self.note_sequence_set)),
                                        dtype=np.float32)
    
    @staticmethod
    def _initialize_note_sequence_set():
        notes = []
        for i in range(128):
            note = m21.note.Note(i)
            notes.append(note.nameWithOctave)
        
        note_sequences = []
        for combo in product(notes, notes):
            note_sequences.append(combo[0] + "|" + combo[1])
        return note_sequences
    
    @staticmethod
    def get_n_note_sequence(part: m21.stream.Part,
                            window_size: int = 2):
        just_notes = part.recurse().notes \
            .getElementsNotOfClass(m21.chord.Chord)
        n_note_sequences = []
        for note_seq in window_gen(just_notes, window_size):
            n_note_sequences.append("|".join([note.nameWithOctave for note in note_seq]))
        return n_note_sequences
    
    @staticmethod
    def sequence_encoder(seq: List[str]):
        d = defaultdict(float)
        for entry in seq:
            d[entry] += 1.
        return d
    
    def _parse_file_as_sequence(self, file_name):
        parsed_file = m21.converter.parse(self.path + file_name)
        n_note_sequence = get_n_note_sequence(parsed_file)
        
        
    def parse_corpus(self):
        for i, file in enumerate(self.files):
            sequence = self._parse_file_as_sequence(file)
            encoded_sequence = self.sequence_encoder(raw_grams)
            for (seq, count) in encoded_sequence.items():
                j = note_sequences.index(seq)
                self.sparse_matrix[i, j] = count

In [174]:
mozart_corpus = MidiFeatureCorpus(path=DATA_DIR)

In [175]:
mozart_corpus.parse_corpus()

In [138]:
notes = []
for i in range(128):
    note = m21.note.Note(i)
    notes.append(note.nameWithOctave)

note_sequences = []
for combo in product(notes, notes):
    note_sequences.append(combo[0] + "|" + combo[1])

In [142]:
note_sequences.index("G4|F5")

8653

In [153]:
S = dok_matrix((6, len(note_sequences)), dtype=np.float32)

In [168]:
for i, raw_grams in enumerate(mozart_grams):
    encoded_sequence = sequence_encoder(raw_grams)
    for (seq, count) in encoded_sequence.items():
        j = note_sequences.index(seq)
        S[i, j] = count